# Сделать грид серч текущей модели

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k
from utils import prefilter_items

In [41]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [42]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [43]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [44]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [45]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [46]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Filter items

In [47]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [48]:
n_items_train_before = data_train['item_id'].nunique()
n_items_test_before = data_test['item_id'].nunique()
n_users_train_before = data_train['user_id'].nunique()
n_users_test_before = data_test['user_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)
data_test_filtered = prefilter_items(data_test, take_n_popular=5000, item_features=item_features)

n_items_train_after = data_train_filtered['item_id'].nunique()
n_items_test_after = data_test_filtered['item_id'].nunique()
n_users_train_after = data_train_filtered['user_id'].nunique()
n_users_test_after = data_test_filtered['user_id'].nunique()
print('Decreased # items_train from {} to {}'.format(n_items_train_before, n_items_train_after))
print('Decreased # items_test from {} to {}'.format(n_items_test_before, n_items_test_after))
print('Decreased # users_train from {} to {}'.format(n_users_train_before, n_users_train_after))
print('Decreased # users_test from {} to {}'.format(n_users_test_before, n_users_test_after))

Decreased # items_train from 86865 to 5001
Decreased # items_test from 24329 to 5001
Decreased # users_train from 2499 to 2497
Decreased # users_test from 2042 to 1990


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [49]:

user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [50]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit
test_user_item_matrix.head(2)

item_id,819210,819308,819423,819487,819840,819845,819978,820321,820347,820352,...,17381676,17381995,17827241,17900997,17901020,17903379,17903423,17959083,18000012,18024556
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [52]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
user_feat.shape

(2497, 7)

In [54]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [55]:
item_feat.shape

(5001, 6)

## Encoding features

In [56]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

## Init model

In [57]:
model = LightFM(no_components=40, # количество факторов (в новой матрице - рисунок наверху)
                 loss='bpr',
                #loss='warp',
                learning_rate=0.05, 
                item_alpha=0.1,  # коеф-ты регуляризации - которые помогают взвесить фичи товаров a1f1 в формуле user_emb
                user_alpha=0.1,  # чтобы не сильно затачиваться под пользователей которые часто покупают
                k=5, # Количнство рекоменд которые мы хотим выдать
                n=15, # Кол-во переборов которые мы хотим взять
                max_sampled=100, # привязан к n и ограничивает 100 товарами которых брать
                random_state=42)

## Train

In [58]:
%%time

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 

CPU times: total: 1min 41s
Wall time: 1min 41s


# Evaluation

In [59]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [60]:
user_emb[0]  # biases

array([-81.08453 ,   0.      ,   0.      , ..., -83.734726, -52.356426,
         0.      ], dtype=float32)

In [61]:
user_emb[0].shape

(2497,)

In [62]:
user_emb[1]  # embeddings

array([[ 3.7863842e-04, -1.7079283e-03, -1.1492267e-04, ...,
        -1.3650634e-04, -3.2811076e-05,  8.9634291e-04],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 6.1603123e-04,  1.0006218e-03,  1.5555976e-03, ...,
         3.1972113e-03,  1.4566761e-03,  2.9825259e-04],
       [-1.5958624e-03,  8.3692661e-03,  1.4935858e-03, ...,
        -4.3662032e-05, -1.2989517e-04, -1.4679842e-02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [63]:
user_emb[1].shape

(2497, 40)

In [64]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [65]:
item_emb[0].shape

(5001,)

In [66]:
item_emb[1].shape

(5001, 40)

# Train precision

In [67]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.21097319

0.21097319 bpr

0.29411295 warp

In [68]:
test_item_ids = np.array([5000, 2000, 3, 200, 1200])

predictions = model.predict(user_ids=10,
                            item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)

In [69]:
predictions

array([-54.762722, -54.897915, -54.25194 , -53.81579 , -54.70555 ],
      dtype=float32)

In [70]:
predictions

array([-54.762722, -54.897915, -54.25194 , -53.81579 , -54.70555 ],
      dtype=float32)

# Test precision

In [71]:
test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_precision

0.0024120605

0.0024120605  bpr

0.0028140703 warp

Neural networks for RS: http://d2l.ai/chapter_recommender-systems/mf.html

LigthFM -> https://arxiv.org/pdf/1507.08439.pdf

https://making.lyst.com/lightfm/docs/home.html


predictions - некоторые числа. 

# Домашнее задание

In [72]:
import datetime
from sklearn.model_selection import ParameterGrid

In [73]:
def grid_search(param_grid):
    time = datetime.datetime.now()
    results = dict()
    grid = list(ParameterGrid(param_grid))
    cnt = 0
    
    for params in grid:
        cnt += 1
        results[cnt] = params
        
        model = LightFM(
            random_state=42,
            **params,
        )
        
        model.fit(
            (sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
            sample_weight=coo_matrix(user_item_matrix),
            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
            epochs=1, 
            num_threads=4,
            verbose=False,
        )

        precision = precision_at_k(
            model,
            csr_matrix(test_user_item_matrix).tocsr(),
            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
            k=5
        )

        results[cnt].update({'precision_at_k': precision.mean()})

        print(f'Iteration {cnt} from {len(grid)} {datetime.datetime.now()} (spent {datetime.datetime.now() - time})')
        print(results[cnt])
        time = datetime.datetime.now()

    return results

In [74]:
param_grid = {
    'no_components': [10, 20, 40, 64],
    'loss': ['bpr', 'warp'],
    'learning_rate': [0.01, 0.05, 0.1], 
    'item_alpha': [0, 0.5],
    'user_alpha': [0, 0.5],
    'max_sampled': [ 5, 10, 15 ],
}

results = grid_search(param_grid)

Iteration 1 from 288 2022-03-05 14:09:09.430828 (spent 0:00:03.345517)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'max_sampled': 5, 'no_components': 10, 'user_alpha': 0, 'precision_at_k': 0.0016080402}
Iteration 2 from 288 2022-03-05 14:09:12.359395 (spent 0:00:02.928567)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'max_sampled': 5, 'no_components': 10, 'user_alpha': 0.5, 'precision_at_k': 0.0014070353}
Iteration 3 from 288 2022-03-05 14:09:17.627636 (spent 0:00:05.267265)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'max_sampled': 5, 'no_components': 20, 'user_alpha': 0, 'precision_at_k': 0.0017085428}
Iteration 4 from 288 2022-03-05 14:09:22.132377 (spent 0:00:04.504741)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'max_sampled': 5, 'no_components': 20, 'user_alpha': 0.5, 'precision_at_k': 0.0027135678}
Iteration 5 from 288 2022-03-05 14:09:31.242357 (spent 0:00:09.109980)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'max_samp

Iteration 39 from 288 2022-03-05 14:13:44.347177 (spent 0:00:13.114537)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'warp', 'max_sampled': 10, 'no_components': 64, 'user_alpha': 0, 'precision_at_k': 0.003115578}
Iteration 40 from 288 2022-03-05 14:13:56.968902 (spent 0:00:12.621725)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'warp', 'max_sampled': 10, 'no_components': 64, 'user_alpha': 0.5, 'precision_at_k': 0.0030150753}
Iteration 41 from 288 2022-03-05 14:14:00.266565 (spent 0:00:03.297663)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'warp', 'max_sampled': 15, 'no_components': 10, 'user_alpha': 0, 'precision_at_k': 0.0017085428}
Iteration 42 from 288 2022-03-05 14:14:03.551786 (spent 0:00:03.284244)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'warp', 'max_sampled': 15, 'no_components': 10, 'user_alpha': 0.5, 'precision_at_k': 0.0016080402}
Iteration 43 from 288 2022-03-05 14:14:08.731293 (spent 0:00:05.179507)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'war

Iteration 77 from 288 2022-03-05 14:18:10.282564 (spent 0:00:08.325769)
{'item_alpha': 0, 'learning_rate': 0.05, 'loss': 'warp', 'max_sampled': 5, 'no_components': 40, 'user_alpha': 0, 'precision_at_k': 0.0022110553}
Iteration 78 from 288 2022-03-05 14:18:18.414027 (spent 0:00:08.131463)
{'item_alpha': 0, 'learning_rate': 0.05, 'loss': 'warp', 'max_sampled': 5, 'no_components': 40, 'user_alpha': 0.5, 'precision_at_k': 0.0015075377}
Iteration 79 from 288 2022-03-05 14:18:30.799156 (spent 0:00:12.385129)
{'item_alpha': 0, 'learning_rate': 0.05, 'loss': 'warp', 'max_sampled': 5, 'no_components': 64, 'user_alpha': 0, 'precision_at_k': 0.0017085428}
Iteration 80 from 288 2022-03-05 14:18:42.867170 (spent 0:00:12.067037)
{'item_alpha': 0, 'learning_rate': 0.05, 'loss': 'warp', 'max_sampled': 5, 'no_components': 64, 'user_alpha': 0.5, 'precision_at_k': 0.0011055276}
Iteration 81 from 288 2022-03-05 14:18:46.099454 (spent 0:00:03.232284)
{'item_alpha': 0, 'learning_rate': 0.05, 'loss': 'warp',

Iteration 115 from 288 2022-03-05 14:22:44.176405 (spent 0:00:04.812265)
{'item_alpha': 0, 'learning_rate': 0.1, 'loss': 'bpr', 'max_sampled': 15, 'no_components': 20, 'user_alpha': 0, 'precision_at_k': 0.002311558}
Iteration 116 from 288 2022-03-05 14:22:48.668503 (spent 0:00:04.492098)
{'item_alpha': 0, 'learning_rate': 0.1, 'loss': 'bpr', 'max_sampled': 15, 'no_components': 20, 'user_alpha': 0.5, 'precision_at_k': 0.0018090453}
Iteration 117 from 288 2022-03-05 14:22:57.212932 (spent 0:00:08.544429)
{'item_alpha': 0, 'learning_rate': 0.1, 'loss': 'bpr', 'max_sampled': 15, 'no_components': 40, 'user_alpha': 0, 'precision_at_k': 0.0075376886}
Iteration 118 from 288 2022-03-05 14:23:04.821586 (spent 0:00:07.608654)
{'item_alpha': 0, 'learning_rate': 0.1, 'loss': 'bpr', 'max_sampled': 15, 'no_components': 40, 'user_alpha': 0.5, 'precision_at_k': 0.0017085428}
Iteration 119 from 288 2022-03-05 14:23:16.260030 (spent 0:00:11.437467)
{'item_alpha': 0, 'learning_rate': 0.1, 'loss': 'bpr', '

Iteration 153 from 288 2022-03-05 14:27:18.404115 (spent 0:00:03.050396)
{'item_alpha': 0.5, 'learning_rate': 0.01, 'loss': 'bpr', 'max_sampled': 10, 'no_components': 10, 'user_alpha': 0, 'precision_at_k': 0.0012060303}
Iteration 154 from 288 2022-03-05 14:27:21.287960 (spent 0:00:02.883845)
{'item_alpha': 0.5, 'learning_rate': 0.01, 'loss': 'bpr', 'max_sampled': 10, 'no_components': 10, 'user_alpha': 0.5, 'precision_at_k': 0.0111557795}
Iteration 155 from 288 2022-03-05 14:27:26.069878 (spent 0:00:04.781918)
{'item_alpha': 0.5, 'learning_rate': 0.01, 'loss': 'bpr', 'max_sampled': 10, 'no_components': 20, 'user_alpha': 0, 'precision_at_k': 0.0059296484}
Iteration 156 from 288 2022-03-05 14:27:30.521734 (spent 0:00:04.451856)
{'item_alpha': 0.5, 'learning_rate': 0.01, 'loss': 'bpr', 'max_sampled': 10, 'no_components': 20, 'user_alpha': 0.5, 'precision_at_k': 0.0025125628}
Iteration 157 from 288 2022-03-05 14:27:38.487313 (spent 0:00:07.965579)
{'item_alpha': 0.5, 'learning_rate': 0.01, 

Iteration 191 from 288 2022-03-05 14:31:38.881352 (spent 0:00:12.739905)
{'item_alpha': 0.5, 'learning_rate': 0.01, 'loss': 'warp', 'max_sampled': 15, 'no_components': 64, 'user_alpha': 0, 'precision_at_k': 0.0016080402}
Iteration 192 from 288 2022-03-05 14:31:51.479539 (spent 0:00:12.598187)
{'item_alpha': 0.5, 'learning_rate': 0.01, 'loss': 'warp', 'max_sampled': 15, 'no_components': 64, 'user_alpha': 0.5, 'precision_at_k': 0.0024120605}
Iteration 193 from 288 2022-03-05 14:31:54.471566 (spent 0:00:02.992027)
{'item_alpha': 0.5, 'learning_rate': 0.05, 'loss': 'bpr', 'max_sampled': 5, 'no_components': 10, 'user_alpha': 0, 'precision_at_k': 0.0010050251}
Iteration 194 from 288 2022-03-05 14:31:57.602267 (spent 0:00:03.130701)
{'item_alpha': 0.5, 'learning_rate': 0.05, 'loss': 'bpr', 'max_sampled': 5, 'no_components': 10, 'user_alpha': 0.5, 'precision_at_k': 0.0026130653}
Iteration 195 from 288 2022-03-05 14:32:02.093230 (spent 0:00:04.490963)
{'item_alpha': 0.5, 'learning_rate': 0.05, 

Iteration 229 from 288 2022-03-05 14:38:32.930985 (spent 0:00:08.159892)
{'item_alpha': 0.5, 'learning_rate': 0.05, 'loss': 'warp', 'max_sampled': 10, 'no_components': 40, 'user_alpha': 0, 'precision_at_k': 0.0020100502}
Iteration 230 from 288 2022-03-05 14:38:41.451742 (spent 0:00:08.520757)
{'item_alpha': 0.5, 'learning_rate': 0.05, 'loss': 'warp', 'max_sampled': 10, 'no_components': 40, 'user_alpha': 0.5, 'precision_at_k': 0.0018090452}
Iteration 231 from 288 2022-03-05 14:38:53.696882 (spent 0:00:12.245140)
{'item_alpha': 0.5, 'learning_rate': 0.05, 'loss': 'warp', 'max_sampled': 10, 'no_components': 64, 'user_alpha': 0, 'precision_at_k': 0.002211055}
Iteration 232 from 288 2022-03-05 14:39:06.464359 (spent 0:00:12.767477)
{'item_alpha': 0.5, 'learning_rate': 0.05, 'loss': 'warp', 'max_sampled': 10, 'no_components': 64, 'user_alpha': 0.5, 'precision_at_k': 0.0024120605}
Iteration 233 from 288 2022-03-05 14:39:09.682903 (spent 0:00:03.218544)
{'item_alpha': 0.5, 'learning_rate': 0.0

Iteration 267 from 288 2022-03-05 14:42:47.685378 (spent 0:00:04.648618)
{'item_alpha': 0.5, 'learning_rate': 0.1, 'loss': 'warp', 'max_sampled': 5, 'no_components': 20, 'user_alpha': 0, 'precision_at_k': 0.010552764}
Iteration 268 from 288 2022-03-05 14:42:52.325705 (spent 0:00:04.640327)
{'item_alpha': 0.5, 'learning_rate': 0.1, 'loss': 'warp', 'max_sampled': 5, 'no_components': 20, 'user_alpha': 0.5, 'precision_at_k': 0.013567839}
Iteration 269 from 288 2022-03-05 14:43:00.187578 (spent 0:00:07.861873)
{'item_alpha': 0.5, 'learning_rate': 0.1, 'loss': 'warp', 'max_sampled': 5, 'no_components': 40, 'user_alpha': 0, 'precision_at_k': 0.010552764}
Iteration 270 from 288 2022-03-05 14:43:08.096061 (spent 0:00:07.908483)
{'item_alpha': 0.5, 'learning_rate': 0.1, 'loss': 'warp', 'max_sampled': 5, 'no_components': 40, 'user_alpha': 0.5, 'precision_at_k': 0.012060301}
Iteration 271 from 288 2022-03-05 14:43:19.878310 (spent 0:00:11.782249)
{'item_alpha': 0.5, 'learning_rate': 0.1, 'loss': '

In [75]:
pd.DataFrame.from_dict(results, orient='index').sort_values(by='precision_at_k', ascending=False)

,item_alpha,learning_rate,loss,max_sampled,no_components,user_alpha,precision_at_k
260,0.5,0.10,bpr,15,20,0.5,0.015779
252,0.5,0.10,bpr,10,20,0.5,0.015779
244,0.5,0.10,bpr,5,20,0.5,0.015779
284,0.5,0.10,warp,15,20,0.5,0.014070
288,0.5,0.10,warp,15,64,0.5,0.013869
...,...,...,...,...,...,...,...
92,0.0,0.05,warp,15,20,0.5,0.000804
36,0.0,0.01,warp,10,20,0.5,0.000804
186,0.5,0.01,warp,15,10,0.5,0.000603
282,0.5,0.10,warp,15,10,0.5,0.000503


При проведениии grid search была достигнута метрика 0.015779 при следующих параметрах:
* no_components = 20
* loss = bpr
* learning_rate = 0.1
* item_alpha = 0.5
* user_alpha = 0.5
* max_sampled = 15
